In [1]:
import pandas as pd
import pickle as pkl

# Global variables

In [2]:
data_folder_name = 'PPG_FieldStudy'

# Functions

In [3]:
def get_script_dir_path():
    script_dir_path = %pwd 
    script_dir_path += '\\'
    return script_dir_path

In [79]:
def standardize_to_700hz(sensor_values, frequency):
    sensor_values_700hz = []
    ratio = 700/frequency
    new_len = int(len(sensor_values) * ratio) # len of the standardized list
    row_index = 0
    for i in range(new_len):
        sensor_values_700hz.append(sensor_values[int(i/ratio)])
        row_index += 1
    return sensor_values_700hz

In [105]:
def multiply_in_list(value, nb):
    new_list = []
    for i in range(nb):
        new_list.append(value)
    return new_list

# 1) Data preparation

## 1.1) Create paths to data

In [4]:
script_dir_path = get_script_dir_path()
print(script_dir_path)

C:\Users\Louis\python_scripts\final_project\python_for_data_analysis\


In [5]:
data_folder_path = script_dir_path + data_folder_name + '\\'
print(data_folder_path)

C:\Users\Louis\python_scripts\final_project\python_for_data_analysis\PPG_FieldStudy\


In [14]:
# Creating path to pkl file for each subject & saving pkl data in list
pkl_path_template = "S{}\\S{}.pkl"
pkl_pd_list = []

for i in range(15):
    path = data_folder_path + pkl_path_template.format(i+1,i+1)
    print(path)
    pkl_pd_list.append(pd.read_pickle(path))

C:\Users\Louis\python_scripts\final_project\python_for_data_analysis\PPG_FieldStudy\S1\S1.pkl
C:\Users\Louis\python_scripts\final_project\python_for_data_analysis\PPG_FieldStudy\S2\S2.pkl
C:\Users\Louis\python_scripts\final_project\python_for_data_analysis\PPG_FieldStudy\S3\S3.pkl
C:\Users\Louis\python_scripts\final_project\python_for_data_analysis\PPG_FieldStudy\S4\S4.pkl
C:\Users\Louis\python_scripts\final_project\python_for_data_analysis\PPG_FieldStudy\S5\S5.pkl
C:\Users\Louis\python_scripts\final_project\python_for_data_analysis\PPG_FieldStudy\S6\S6.pkl
C:\Users\Louis\python_scripts\final_project\python_for_data_analysis\PPG_FieldStudy\S7\S7.pkl
C:\Users\Louis\python_scripts\final_project\python_for_data_analysis\PPG_FieldStudy\S8\S8.pkl
C:\Users\Louis\python_scripts\final_project\python_for_data_analysis\PPG_FieldStudy\S9\S9.pkl
C:\Users\Louis\python_scripts\final_project\python_for_data_analysis\PPG_FieldStudy\S10\S10.pkl
C:\Users\Louis\python_scripts\final_project\python_for_dat

## 1.2) Create pandas dataframe from data

In [119]:
def create_pd_dataframe(pkl_data):
    # Capteur Empatica E4
    wrist_ACC = pkl_data['signal']['wrist']['ACC'] # 32 Hz
    wrist_BVP = pkl_data['signal']['wrist']['BVP'] # 64 Hz
    wrist_EDA = pkl_data['signal']['wrist']['EDA'] # 4 Hz
    wrist_TEMP = pkl_data['signal']['wrist']['TEMP'] # 4 Hz

    # Capteur RespiBAN
    chest_ACC = pkl_data['signal']['chest']['ACC'] # 700 Hz
    chest_ECG = pkl_data['signal']['chest']['ECG'] # 700 Hz
    chest_resp = pkl_data['signal']['chest']['Resp'] # 700 Hz

    print('len wrist_ACC : ' + str(len(wrist_ACC)))
    print('len wrist_BVP : ' + str(len(wrist_BVP)))
    print('len wrist_EDA : ' + str(len(wrist_EDA)))
    print('len wrist_TEMP : ' + str(len(wrist_TEMP)))
    print()
    print('len chest_ACC : ' + str(len(chest_ACC)))
    print('len chest_ECG : ' + str(len(chest_ECG)))
    print('len chest_resp : ' + str(len(chest_resp)))
    print()


    # Standardize every values from empatica E4 sensor to 700hz format
    wrist_ACC_700hz = standardize_to_700hz(wrist_ACC, 32)
    wrist_BVP_700hz = standardize_to_700hz(wrist_BVP, 64)
    wrist_EDA_700hz = standardize_to_700hz(wrist_EDA, 4)
    wrist_TEMP_700hz = standardize_to_700hz(wrist_TEMP, 4)

    print('new len wrist_ACC 700hz : ' + str(len(wrist_ACC_700hz)))
    print('new len wrist_BVP 700hz : ' + str(len(wrist_BVP_700hz)))
    print('new len wrist_EDA 700hz : ' + str(len(wrist_EDA_700hz)))
    print('new len wrist_TEMP 700hz : ' + str(len(wrist_TEMP_700hz)))
    print()

    # Activity
    activity = pkl_data['activity']
    print('len activity : ' + str(len(activity)))

    # Standardize activity values to 700hz format
    activity_700hz = standardize_to_700hz(activity, 4)
    print('new len activity : ' + str(len(activity_700hz)))
    print()

    # subjects infos
    age = pkl_data['questionnaire']['AGE']
    gender = pkl_data['questionnaire']['Gender']
    height = pkl_data['questionnaire']['HEIGHT']
    weight = pkl_data['questionnaire']['WEIGHT']
    skin = pkl_data['questionnaire']['SKIN']
    sport = pkl_data['questionnaire']['SPORT']

    print('age : ' + str(age))
    print('gender : ' + gender)
    print('height : ' + str(height))
    print('weight : ' + str(weight))
    print('skin : ' + str(skin))
    print('sport : ' + str(sport))
    
    # replace gender per a number
    if (gender == ' m'):
        gender = 0
    else:
        gender = 1
    
    print('new gender value : ' + str(gender))
    
    # create lists for subjects infos by just multiplying value by len(chest_resp)
    target_len = len(chest_resp)
    
    age_list = multiply_in_list(age, target_len)
    gender_list = multiply_in_list(gender, target_len)
    height_list = multiply_in_list(height, target_len)
    weight_list = multiply_in_list(weight, target_len)
    skin_list = multiply_in_list(skin, target_len)
    sport_list = multiply_in_list(sport, target_len)

    
    # Create dataframe
    
    


In [121]:
dataframe = create_pd_dataframe(pkl_pd_list[1])


len wrist_ACC : 262560
len wrist_BVP : 525120
len wrist_EDA : 32820
len wrist_TEMP : 32820

len chest_ACC : 5743500
len chest_ECG : 5743500
len chest_resp : 5743500

new len wrist_ACC 700hz : 5743500
new len wrist_BVP 700hz : 5743500
new len wrist_EDA 700hz : 5743500
new len wrist_TEMP 700hz : 5743500

len activity : 32820
new len activity : 5743500

age : 28
gender :  m
height : 189.0
weight : 80.0
skin : 3
sport : 5
new gender value : 0


In [112]:
new_list = multiply_in_list(age, len(chest_resp))

6448400
